In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin


from sklearn.linear_model import Lasso, Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb
import lightgbm as lgb

pd.set_option('max_columns', 200)
pd.set_option('max_rows', 80)

In [2]:
def get_coef(pipe):
    imp = pipe.steps[-1][1].coef_.tolist()
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result['abs_res'] = abs(result['score'])
    result = result.sort_values(by=['abs_res'],ascending=False)
    del result['abs_res']
    return result


def get_feature_importance(pipe):
    imp = pipe.steps[-1][1].feature_importances_.tolist() #it's a pipeline
    feats = pipe.steps[-2][1].get_feature_names()
    result = pd.DataFrame({'feat':feats,'score':imp})
    result = result.sort_values(by=['score'],ascending=False)
    return result

def _plot_diagonal(ax):
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    low = min(xmin, xmax)
    high = max(xmin, xmax)
    scl = (high - low) / 100
    
    line = pd.DataFrame({'x': np.arange(low, high ,scl), # small hack for a diagonal line
                         'y': np.arange(low, high ,scl)})
    ax.plot(line.x, line.y, color='black', linestyle='--')
    
    return ax


def plot_predictions(data, true_label, pred_label, feature=None, hue=None, legend=False, savename='test.png'):
    '''
    Plot prediction vs true label or a specific feature. It also plots the residuals plot
    '''
    
    tmp = data.copy()
    tmp['Prediction'] = pred_label
    tmp['True Label'] = true_label
    tmp['Residual'] = tmp['True Label'] - tmp['Prediction']
    
    diag = False
    alpha = 0.7
    label = ''
    
    fig, ax = plt.subplots(1,2, figsize=(15,6))
    
    if feature is None:
        feature = 'True Label'
        diag = True
    else:
        legend = 'full'
        sns.scatterplot(x=feature, y='True Label', data=tmp, ax=ax[0], label='True',
                         hue=hue, legend=legend, alpha=alpha)
        label = 'Predicted'
        alpha = 0.4

    sns.scatterplot(x=feature, y='Prediction', data=tmp, ax=ax[0], label=label,
                         hue=hue, legend=legend, alpha=alpha)
    if diag:
        ax[0] = _plot_diagonal(ax[0])
    
    sns.scatterplot(x=feature, y='Residual', data=tmp, ax=ax[1], 
                    hue=hue, legend=legend, alpha=0.7)
    ax[1].axhline(y=0, color='r', linestyle='--')
    
    ax[0].set_title(f'{feature} vs Predictions')
    ax[1].set_title(f'{feature} vs Residuals')
    
    if not savename.endswith('.png'):
        savename += '.png'
    plt.savefig('../plots/' + savename)
    plt.close()

In [3]:
class df_imputer(TransformerMixin, BaseEstimator):
    '''
    Just a wrapper for the SimpleImputer that keeps the dataframe structure
    '''
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.imp = None
        self.statistics_ = None

    def fit(self, X, y=None):
        self.imp = SimpleImputer(strategy=self.strategy)
        self.imp.fit(X)
        self.statistics_ = pd.Series(self.imp.statistics_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Ximp = self.imp.transform(X)
        Xfilled = pd.DataFrame(Ximp, index=X.index, columns=X.columns)
        return Xfilled

    
class df_scaler(TransformerMixin, BaseEstimator):
    '''
    Wrapper of StandardScaler or RobustScaler
    '''
    def __init__(self, method='standard'):
        self.scl = None
        self.scale_ = None
        self.method = method
        if self.method == 'sdandard':
            self.mean_ = None
        elif method == 'robust':
            self.center_ = None
        self.columns = None  # this is useful when it is the last step of a pipeline before the model

    def fit(self, X, y=None):
        if self.method == 'standard':
            self.scl = StandardScaler()
            self.scl.fit(X)
            self.mean_ = pd.Series(self.scl.mean_, index=X.columns)
        elif self.method == 'robust':
            self.scl = RobustScaler()
            self.scl.fit(X)
            self.center_ = pd.Series(self.scl.center_, index=X.columns)
        self.scale_ = pd.Series(self.scl.scale_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xscl = self.scl.transform(X)
        Xscaled = pd.DataFrame(Xscl, index=X.index, columns=X.columns)
        self.columns = X.columns
        return Xscaled

    def get_feature_names(self):
        return list(self.columns)


class dummify(TransformerMixin, BaseEstimator):
    '''
    Wrapper for get dummies
    Via match_cols, it is possible to ask the transformer to make sure that all the dummies are there
    Missing dummies are introduced with a column of 0's
    Extra dummies are dropped
    '''
    def __init__(self, drop_first=False, match_cols=True):
        self.drop_first = drop_first
        self.columns = []  # useful to well behave with FeatureUnion
        self.match_cols = match_cols

    def fit(self, X, y=None):
        return self
    
    def match_columns(self, X):
        miss_train = list(set(X.columns) - set(self.columns))
        miss_test = list(set(self.columns) - set(X.columns))
        
        err = 0
        
        if len(miss_test) > 0:
            for col in miss_test:
                X[col] = 0  # insert a column for the missing dummy
                err += 1
        if len(miss_train) > 0:
            for col in miss_train:
                del X[col]  # delete the column of the extra dummy
                err += 1
                
        if err > 0:
            warnings.warn('The dummies in this set do not match the ones in the train set, we corrected the issue.',
                         UserWarning)
            
        return X[self.columns]
        
        
    def transform(self, X):
        X = pd.get_dummies(X, drop_first=self.drop_first)
        if (len(self.columns) > 0):
            if self.match_cols:
                X = self.match_columns(X)
        else:
            self.columns = X.columns
        return X
    
    def get_features_name(self):
        return list(self.columns)

In [4]:
class transformation(TransformerMixin, BaseEstimator):
    def __init__(self, mean_weight=10):
        self.columns = None

        
    def fit(self, X, y=None):
        return self
    
    
    def stats_by_play(self, data):
        avg_by_play = data.groupby(['PlayId', 
                                    'Team', 
                                    'offense_team'], as_index=False)[['PlayerHeight', 
                                                                      'PlayerWeight',
                                                                      'age',
                                                                      'S', 'A']].mean()
        spread = data.groupby(['PlayId', 
                               'Team', 
                               'offense_team'])[['X', 'Y']].std().reset_index()
        tot_momentum = data.groupby(['PlayId', 
                                     'Team', 
                                     'offense_team'], as_index=False)[['X_speed', 'Y_speed',
                                                                       'PlayerWeight',
                                                                       'X_acceleration','Y_acceleration']].sum()
        
        tot_momentum['x_momentum'] = tot_momentum['X_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['y_momentum'] = tot_momentum['Y_speed'] * tot_momentum['PlayerWeight']
        tot_momentum['x_force'] = tot_momentum['X_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum['y_force'] = tot_momentum['Y_acceleration'] * tot_momentum['PlayerWeight']
        tot_momentum.drop(['X_speed', 'Y_speed',
                           'PlayerWeight',  
                           'X_acceleration','Y_acceleration'], axis=1, inplace=True)

        avg_by_play = pd.merge(avg_by_play, tot_momentum, on=['PlayId', 'Team', 'offense_team'])
        avg_by_play = pd.merge(avg_by_play, spread, on=['PlayId', 'Team', 'offense_team'])

        poss_team = avg_by_play[avg_by_play.Team == avg_by_play.offense_team].copy()
        def_team = avg_by_play[avg_by_play.Team != avg_by_play.offense_team].copy()

        poss_team.rename(columns={'PlayerHeight': 'poss_avg_height', 
                                  'PlayerWeight': 'poss_avg_weight',
                                  'age': 'poss_avg_age',
                                  'X': 'poss_std_X',
                                  'Y': 'poss_std_Y',
                                  'S': 'poss_avg_S', 
                                  'A': 'poss_avg_A', 
                                  'x_momentum': 'poss_x_momentum', 
                                  'y_momentum': 'poss_y_momentum', 
                                  'x_force': 'poss_x_force', 
                                  'y_force': 'poss_y_force'}, inplace=True)
        def_team.rename(columns={'PlayerHeight': 'def_avg_height', 
                                  'PlayerWeight': 'def_avg_weight', 
                                  'age': 'def_avg_age',
                                  'X': 'def_std_X',
                                  'Y': 'def_std_Y',
                                  'S': 'def_avg_S', 
                                  'A': 'def_avg_A',
                                  'x_momentum': 'def_x_momentum', 
                                  'y_momentum': 'def_y_momentum', 
                                  'x_force': 'def_x_force', 
                                  'y_force': 'def_y_force'}, inplace=True)

        avg_by_play = pd.merge(poss_team.drop('Team', axis=1), 
                               def_team.drop('Team', axis=1), on=['PlayId', 'offense_team'])
        
        avg_by_play['tot_x_momenumt'] = avg_by_play['poss_x_momentum'] - avg_by_play['def_x_momentum']
        avg_by_play['tot_x_force'] = avg_by_play['poss_x_force'] - avg_by_play['def_x_force']
        avg_by_play['height_diff'] = avg_by_play['poss_avg_height'] - avg_by_play['def_avg_height']
        avg_by_play['weight_diff'] = avg_by_play['poss_avg_weight'] - avg_by_play['def_avg_weight']
        avg_by_play['age_diff'] = avg_by_play['poss_avg_age'] - avg_by_play['def_avg_age']
        avg_by_play['X_diff'] = avg_by_play['poss_std_X'] - avg_by_play['def_std_X']
        avg_by_play['Y_diff'] = avg_by_play['poss_std_Y'] - avg_by_play['def_std_Y']

        return avg_by_play
    
    
    def process_play(self, X):
        cols_by_play = ['GameId', 'PlayId', 'YardLine', 
                'Quarter', 'GameClock', 'Down', 'Distance',
                'OffenseFormation', 'DefendersInTheBox',  
                'Location', 'StadiumType', 'Turf', 
                'GameWeather','Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 
                'PlayDirection', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay']
        train_play = X[cols_by_play].drop_duplicates()
        avg_by_play = self.stats_by_play(X)
        train_play = pd.merge(train_play, avg_by_play.drop('offense_team', axis=1), on=['PlayId'])

        return train_play
    
    
    def dropper(self, X):
        cleaned = X.copy()
        to_drop = ['GameId', 'WindDirection', 'WindSpeed', 'GameWeather', 
                    'PlayDirection', 'StadiumType', 'Turf', 'Location', 
                   'GameClock', 'distance_from_ball', 'Quarter', 'Down', 
                   'OffenseFormation', 'Temperature', 'Humidity', 
                   'HomeScoreBeforePlay','VisitorScoreBeforePlay', 
                   'Dis', 'Dir', 'Yards', 'Distance', 'PlayId', 'X', 'Y']
        for col in to_drop:
            try:
                del cleaned[col]
            except KeyError:
                pass
        
        return cleaned
    
    
    def transform(self, X, y=None):
        train_play = self.process_play(X)
        carriers = X[X.has_ball].copy()

        to_drop = ['GameId', 'NflId', 'Team', 'Orientation','YardLine', 'Quarter', 'GameClock', 'PossessionTeam',
           'Down', 'FieldPosition', 'HomeScoreBeforePlay',
           'VisitorScoreBeforePlay', 'NflIdRusher', 'OffensePersonnel','DefensePersonnel',
               'PlayDirection', 'Position', 'HomeTeamAbbr',
           'VisitorTeamAbbr', 'Location', 'StadiumType', 'GameWeather',
           'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'to_left',
           'has_ball', 'offense_team', 'Distance',
           'OffenseFormation', 'DefendersInTheBox', 'Turf']

        carriers.drop(to_drop, axis=1, inplace=True)

        full_train = pd.merge(carriers, train_play, on='PlayId')
        
        full_train = self.dropper(full_train)
        
        self.columns = full_train.columns

        return full_train
    
    
    def get_features_name(self):
        return self.columns

In [5]:
df_train = pd.read_csv('../data_processed/train_processed.csv', dtype={'WindSpeed': 'object'})

In [6]:
transf_pipe = Pipeline([('trsf', transformation())])

full_train = transf_pipe.fit_transform(df_train)

full_train.head()

,S,A,PlayerHeight,PlayerWeight,from_yardline,X_speed,Y_speed,X_acceleration,Y_acceleration,age,closest_opponent,opponents_in_6,teammates_in_6,YardLine,DefendersInTheBox,poss_avg_height,poss_avg_weight,poss_avg_age,poss_avg_S,poss_avg_A,poss_x_momentum,poss_y_momentum,poss_x_force,poss_y_force,poss_std_X,poss_std_Y,def_avg_height,def_avg_weight,def_avg_age,def_avg_S,def_avg_A,def_x_momentum,def_y_momentum,def_x_force,def_y_force,def_std_X,def_std_Y,tot_x_momenumt,tot_x_force,height_diff,weight_diff,age_diff,X_diff,Y_diff
0,3.63,3.35,70,205,3.75,1.491487,3.309436,1.376441,3.054163,9349,4.593310,3.0,7.0,45,6.0,74.727273,259.181818,10374.454545,2.106364,1.358182,30394.065885,42824.323203,14808.310917,29092.727988,1.772665,5.855606,73.636364,233.545455,10101.272727,1.314545,1.025455,21967.405111,4867.003640,11406.041625,-783.137540,5.294079,7.014714,8426.660774,3402.269293,1.090909,25.636364,273.181818,-3.521414,-1.159107
1,3.06,2.41,70,205,4.07,-2.055465,2.266862,-1.618847,1.785339,9349,4.287773,3.0,7.0,53,6.0,74.727273,259.181818,10374.454545,2.094545,1.541818,5361.669172,36553.979512,1503.788380,27039.089576,2.001380,6.932502,73.636364,233.545455,10101.272727,1.639091,1.592727,-3627.891830,6256.141431,-2706.292685,4533.856018,5.406292,7.190716,8989.561002,4210.081065,1.090909,25.636364,273.181818,-3.404912,-0.258215
2,5.77,2.42,70,205,3.66,4.290640,3.857889,1.799541,1.618040,9349,4.221670,2.0,6.0,75,7.0,74.727273,259.181818,10374.454545,3.682727,1.419091,86277.195695,48107.583525,33664.932926,23232.912519,2.115286,6.044208,73.636364,233.545455,10101.272727,3.244545,2.092727,82977.152392,12860.033647,52837.531067,8496.045083,4.720893,6.577620,3300.043303,-19172.598141,1.090909,25.636364,273.181818,-2.605606,-0.533412
3,4.45,3.20,71,210,3.53,-0.421875,4.429957,-0.303371,3.185587,9808,4.528002,7.0,9.0,108,9.0,76.181818,282.545455,10320.636364,2.141818,0.880909,-39812.029305,39225.794933,-11636.814745,16186.365888,1.898686,3.154869,73.727273,257.454545,10390.181818,1.555455,1.293636,-37637.593645,-4639.830146,-26960.759277,-9328.539063,0.962418,4.505029,-2174.435660,15323.944532,2.454545,25.090909,-69.545455,0.936268,-1.350160
4,3.90,2.53,71,216,5.01,-3.613974,1.466013,-2.344450,0.951029,8069,4.288088,3.0,6.0,35,7.0,76.909091,268.454545,9732.818182,2.644545,1.620000,-31427.161922,20004.068984,-13921.201825,13577.076191,2.056883,7.423977,73.181818,242.454545,9967.181818,2.322727,2.121818,-54049.085285,-11151.985380,-50459.233839,-5737.783040,5.391251,7.864325,22621.923362,36538.032014,3.727273,26.000000,-234.363636,-3.334368,-0.440348


In [7]:
def create_targets(data):
    unique_plays = data[['PlayId', 'Yards']].drop_duplicates()
    yards = unique_plays['Yards'].reset_index(drop=True)
    
    y = np.zeros((yards.shape[0], 199))
    for idx, target in enumerate(list(yards)):
        y[idx][99 + target] = 1
    
    return yards, y


def crps_score(y_prediction, y_valid, shape):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

In [8]:
yards, y = create_targets(df_train)

y[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
folds = KFold(5, shuffle=True, random_state=541)

plays = df_train[df_train.has_ball][['PlayId', 
                                     'YardLine', 
                                     'from_yardline', 
                                     'X']].drop_duplicates().reset_index(drop=True)

In [10]:
def cv_score(df_train, y_train, kfolds, pipeline, imp_coef=False):
    oof = np.zeros([y_train.shape[0], y_train.shape[1]])
    
    train = df_train.copy()
    
    feat_df = pd.DataFrame()
    
    for n_fold, (train_index, test_index) in enumerate(kfolds.split(train.values)):
            
        trn_data = train.iloc[train_index][:]
        val_data = train.iloc[test_index][:]
        
        trn_target = y_train[train_index]
        val_target = y_train[test_index]
        
        pipeline.fit(trn_data, trn_target)

        oof[test_index, :] = pipeline.predict(val_data)

        if imp_coef:
            try:
                fold_df = get_coef(pipeline)
            except AttributeError:
                fold_df = get_feature_importance(pipeline)
                
            fold_df['fold'] = n_fold + 1
            feat_df = pd.concat([feat_df, fold_df], axis=0)
       
    if imp_coef:
        feat_df = feat_df.groupby('feat')['score'].agg(['mean', 'std'])
        feat_df['abs_sco'] = (abs(feat_df['mean']))
        feat_df = feat_df.sort_values(by=['abs_sco'],ascending=False)
        del feat_df['abs_sco']
        return oof, feat_df
    else:    
        return oof

In [11]:
models = [('ridge', Ridge()), 
          ('forest', RandomForestRegressor(n_estimators=100, n_jobs=-1,  
                                           criterion='mse', max_features='auto'))]#, 
          #('xtree', ExtraTreesRegressor(n_estimators=100, n_jobs=-1, 
          #                              criterion='mse', max_features=0.8))]#, 
          #('xgb', xgb.XGBRegressor(n_estimators=600, objective='reg:squarederror', n_jobs=-1)), 
          #('lgb', lgb.LGBMRegressor(n_estimators=600, n_jobs=-1))]

mod_name = []
rmse_train_simple = []
mae_train_simple = []
crps_train_simple = []
oofs = {}

avg_preds = np.zeros((yards.shape[0], 199))


for model in models:
    
    train = full_train.copy()
    print(model[0])
    mod_name.append(model[0])
    
    model_pipe = Pipeline([('dummifier', dummify(drop_first=True)),
                           ('Imputer', df_imputer()),
                           ('scl', df_scaler(method='standard'))] + [model])
            
    %time inf_preds= cv_score(train, y, folds, model_pipe)
    
    avg_preds += inf_preds / 3
    
    crps = crps_score(inf_preds, y, train.shape[0])
    
    oofs[model[0]] = inf_preds
    
    inf_preds = inf_preds.argmax(axis=1) - 99

    rmse_train_simple.append(np.sqrt(mean_squared_error(yards, inf_preds)))
    mae_train_simple.append(mean_absolute_error(yards, inf_preds))
    crps_train_simple.append(crps)
    
    print(f'\tTrain set RMSE: {round(np.sqrt(mean_squared_error(yards, inf_preds)), 4)}')
    print(f'\tTrain set MAE: {round(mean_absolute_error(yards, inf_preds), 4)}')
    print(f'\tTrain set CRPS: {round(crps, 4)}')
    
    plot_predictions(train, yards, inf_preds, savename='../plots/'+model[0]+'.png')
    
    print('- -'*20)

    
crps = crps_score(avg_preds, y, train.shape[0])

oofs['avg'] = avg_preds

avg_preds = avg_preds.argmax(axis=1) - 99

mod_name.append('avg')
rmse_train_simple.append(np.sqrt(mean_squared_error(yards, avg_preds)))
mae_train_simple.append(mean_absolute_error(yards, avg_preds))
crps_train_simple.append(crps)

print(f'\tTrain set RMSE average: {round(np.sqrt(mean_squared_error(yards, avg_preds)), 4)}')
print(f'\tTrain set MAE average: {round(mean_absolute_error(yards, avg_preds), 4)}')
print(f'\tTrain set CRPS average: {round(crps, 4)}')

print('_'*60)
print('\n')
    
    
results = pd.DataFrame({'model_name': mod_name, 
                        'rmse_train': rmse_train_simple, 
                        'mae_train': mae_train_simple, 
                        'crps_train': crps_train_simple})

results 

ridge
CPU times: user 2.3 s, sys: 1.56 s, total: 3.86 s
Wall time: 868 ms
	Train set RMSE: 6.6825
	Train set MAE: 3.6144
	Train set CRPS: 0.013
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
forest
CPU times: user 36min 50s, sys: 2.87 s, total: 36min 53s
Wall time: 6min 26s
	Train set RMSE: 6.6739
	Train set MAE: 3.6119
	Train set CRPS: 0.013
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
	Train set RMSE average: 6.6237
	Train set MAE average: 3.55
	Train set CRPS average: 0.065
____________________________________________________________




,model_name,rmse_train,mae_train,crps_train
0,ridge,6.682540,3.614432,0.013030
1,forest,6.673922,3.611886,0.012952
2,avg,6.623714,3.549998,0.064952


In [16]:
plot_predictions(train, yards, avg_preds, savename='../plots/avg.png')

In [24]:
tmp = oofs['ridge'].copy()

tmp

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         8.91683381e-05,  0.00000000e+00, -1.19422384e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.99814170e-05,  0.00000000e+00,  3.43384170e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.98713980e-05,  0.00000000e+00, -2.36919351e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.47091326e-04,  0.00000000e+00, -6.96067131e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.04025765e-04,  0.00000000e+00,  2.05157211e-04]])

In [13]:
def post_model(mod_plays, pred, data):
    result = pred.copy()
    # only non negative values
    result[result < 0] = 0 
    return result

In [14]:
post_model(plays, tmp, train)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.91683381e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.43384170e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.98713980e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.04025765e-04, 0.00000000e+00, 2.05157211e-04]])